In [15]:
import pandas as pd
from glob import glob
import xml.etree.ElementTree as ET
import json

In [16]:
old_path = "./data/カタログデータ - 新規画像撮影の対応表.csv"

old_df = pd.read_csv(old_path)

old_map = {}

old_map_extra = {}

for index, row in old_df.iterrows():
    value = row["画像一覧"]
    if pd.isna(value):
        continue

    id_new = row["dcterms:identifer"]

    if pd.isna(id_new):
        continue
    
    old_map[row["現在のアイテムID"]] = id_new

# old_map

with open("./tmp/old_map.json", "w") as f:
    json.dump(old_map, f)


In [22]:
files = glob("../docs/tei3/*.xml")

files.sort()

errors = []

for file in files:
    old_id = file.split("/")[-1].split(".")[0]

    if old_id.startswith("Tursko"):
        continue

    if old_id not in old_map:
        # print(old_id, old_map[old_id])
        errors.append(old_id)

        continue

    new_id = old_map[old_id]

    tree = ET.parse(file)
    root = tree.getroot()

    root.find(".//{http://www.tei-c.org/ns/1.0}title").text = new_id

    tree.write(file)

    break




In [24]:
from tqdm import tqdm

In [26]:
from xml.dom import minidom
import re

In [27]:

def prettify(rough_string):
    reparsed = minidom.parseString(rough_string)
    xml_declaration = '''<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml"
	schematypens="http://purl.oclc.org/dsdl/schematron"?>'''

    # まずprettyXMLを取得
    pretty = reparsed.toprettyxml(indent="\t")
    
    # XML宣言を置き換え
    pretty = re.sub(r'<\?xml.*?\?>', xml_declaration, pretty, flags=re.DOTALL)
    
    # 整形処理
    pretty = re.sub(r"[\t ]+\n", "", pretty)  # インデント後の不要な改行を削除
    pretty = pretty.replace(">\n\n\t<", ">\n\t<")  # 不要な空行を削除
    pretty = re.sub(r"\n\s*\n", "\n", pretty)  # 連続した改行（空白行を含む）を単一の改行に置換
    
    # エスケープされた引用符を元に戻す
    pretty = pretty.replace("&quot;", "\"")
    
    return pretty

In [28]:
files = glob("../docs/tei3/*.xml")

files.sort()

errors = []

ET.register_namespace('', "http://www.tei-c.org/ns/1.0")

for file in tqdm(files):
    file_id = file.split("/")[-1].split(".")[0]

    

    tree = ET.parse(file)
    root = tree.getroot()

    root.find(".//{http://www.tei-c.org/ns/1.0}title").text = file_id

    with open(file, "w") as f:
        f.write(prettify(ET.tostring(root, encoding="utf-8").decode("utf-8")))





100%|██████████| 1049/1049 [00:02<00:00, 521.14it/s]
